In [26]:
# SETTINGS FOR ADD DATA
# Общее количество действий каждого вида в мобильных событиях за все время
EACH_EVENT_TOTAL_SUM = True 
# Общее количество всех действий в мобильных за все время
ALL_EVENTS_TOTAL_SUM = True 
# Общее количество уникальных дней, проведенных в мобильных событиях
EVENTS_DAYS_TOTAL_SUM = True
# Общее количество уникальных месяцев, проведенных в мобильных событиях
EVENTS_MONTHS_TOTAL_SUM = True
# Общее количество выданных промокодов за октябрь
PROMO_OCTOBER_TOTAL_SUM= True
# Применял ли клиент хоть раз промокод за все время
APPLIED_PROMO_TOTAL_ONCE = True
# Сумма потраченная на заказы за все время 
MONEY_SPENT_TOTAL_SUM = True
# Сумма уникальных продуктов, приобретенных за все время
UNIQUE_PRODUCTS_TOTAL_SUM = True    
# Сумма заказов каждого вида типа продукта за все время
ORDER_TYPE_TOTAL_SUM = True
# Количество дней, которое прошло с момента последнего заказа до 1 ноября (-1 если нет заказов)
DAYS_SINCE_LAST_ORDER = True
# Сумма уникальных категорий продуктов, выбранных за все время
UNIQUE_CATEGORIES_TOTAL_SUM = True
# Выгодность промокода - скидка / цену
PROMO_VALUE = True
# Количество заказов с промокодом / общее количество заказов за все время
PROMO_USAGE_RATIO_TOTAL_SUM = True
# Подсчет заказов с состоянием 11 и 4 за все время
EACH_ORDER_STATE_TOTAL_SUM = True
# Количество уникальных устройств, используемое за все время в мобильных событиях
DEVICE_DIVERSITY = True
# Сумма уникальных секторов доставки за все время
UNIQUE_SECTOR_TOTAL_SUM = True
# Среднее время (в днях) между заказами за все время (-1 иначе) 
AVG_TIME_BETWEEN_ORDERS = True
# Сумма заказов по дням недели за все время
ORDERS_WEEKDAY_TOTAL_SUM = True
# Сумма уникальных способов оплаты (3 колонки - 3 способа), примененных за все время
ORDER_PAYMENT_TYPE_SUM = True
# Максимальная сумма заказа (продукта) за все время
MAX_ORDER_VALUE = True
# Средняя скидка при заказе за все время
AVG_DISCOUNT = True
# Разница в днях между первым и последним заказом за все время
FIRST_LAST_ORDER_DIFF = True
# Сумма всех скидок, выданных за октябрь
PROMO_DISCOUNT_OCTOBER_SUM = True
# Количество заказов с применением промокода / общее количество заказов (за октябрь)
PROMO_USAGE_RATIO_OCTOBER = True
# Сумма уникальных id пиццерий за все время
UNIQUE_PIZZERIA_TOTAL_SUM = True
# Подсчет количества заказов по категориям продуктов за все время
CATEGORY_USAGE = True
# Подсчет количества заказов в разное время суток за все время
HOURS_ORDERS = True
# Общая сумма скидок для всех заказов за все время
TOTAL_DISCOUNTS = True
# Сумма уникальных VisitToken за все время
VISIT_TOKENS = True

In [27]:
import pandas as pd
import numpy as np

In [28]:
cl_pr_oct = pd.read_csv('clients_promo_october.csv')

mobile_events = pd.read_csv('mobile_events.csv')

orders = pd.read_csv('orders.csv')

test = pd.read_csv('test.csv')
test['OrderType'] = test['OrderType'].apply(lambda x: 1 if '1' in x or '2' in x or '3' in x else 0)
test.drop(columns=['LocalBeginDate', 'LocalEndDate'], inplace=True) 

train = pd.read_csv('train_target.csv')
train['OrderType'] = train['OrderType'].apply(lambda x: 1 if '1' in x or '2' in x or '3' in x else 0)
train.drop(columns=['LocalBeginDate', 'LocalEndDate'], inplace=True)

In [ ]:
if EACH_EVENT_TOTAL_SUM:
    # Создаем сводную таблицу с количеством каждого события для каждого клиента
    event_counts = mobile_events.pivot_table(index='ClientUUId', 
                                             columns='EventName', 
                                             aggfunc='size', 
                                             fill_value=0)
    # Преобразуем MultiIndex колонок в обычный индекс
    event_counts.columns = [f"{col}_total_sum" for col in event_counts.columns]
    
    # Объединение признаков с `train` и `test`
    train = train.merge(event_counts, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(event_counts, on='ClientUUId', how='left').fillna(-1)


if ALL_EVENTS_TOTAL_SUM:
    # Подсчет общего количества событий для каждого клиента
    total_events = mobile_events['ClientUUId'].value_counts()
    total_events.rename('all_events_total_sum', inplace=True)
    
    train = train.merge(total_events, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(total_events, on='ClientUUId', how='left').fillna(-1)


if EVENTS_DAYS_TOTAL_SUM:
    # Извлечение даты из Timestamp и подсчет количества уникальных дат (дней) посещений
    mobile_events['VisitDate'] = pd.to_datetime(mobile_events['Timestamp']).dt.date
    # Группировка по ClientUUId и подсчет количества уникальных дней посещений
    unique_visits_per_client = mobile_events.groupby('ClientUUId')['VisitDate'].nunique().reset_index()
    # Переименование колонок для ясности
    unique_visits_per_client.columns = ['ClientUUId', 'event_days_total_sum']
    
    train = train.merge(unique_visits_per_client, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_visits_per_client, on='ClientUUId', how='left').fillna(-1)

if EVENTS_MONTHS_TOTAL_SUM:
    # Извлечение месяца из Timestamp и подсчет количества уникальных месяцев посещений
    mobile_events['VisitMonth'] = pd.to_datetime(mobile_events['Timestamp']).dt.to_period('M')
    unique_visit_months_per_client = mobile_events.groupby('ClientUUId')['VisitMonth'].nunique().reset_index()
    unique_visit_months_per_client.columns = ['ClientUUId', 'events_months_total_sum']
    
    train = train.merge(unique_visit_months_per_client, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_visit_months_per_client, on='ClientUUId', how='left').fillna(-1)
    
    
if PROMO_OCTOBER_TOTAL_SUM:
    # Подсчет количества промо для каждого клиента
    promo_amount = cl_pr_oct['ClientUUId'].value_counts()
    promo_amount.rename('promo_october_total_sum', inplace=True)
    
    
    # Объединение признаков с `train` и `test`
    train = train.merge(promo_amount, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(promo_amount, on='ClientUUId', how='left').fillna(-1)


if APPLIED_PROMO_TOTAL_ONCE:
    applied_promo = orders[orders['apply_promo'] == 1]
    applied_promo = applied_promo['ClientUUId'].unique()
    
    train['applied_promo_total_once'] = train['ClientUUId'].isin(applied_promo)
    test['applied_promo_total_once'] = test['ClientUUId'].isin(applied_promo)


if MONEY_SPENT_TOTAL_SUM:
    # Группировка по ClientUUId и подсчет суммы ProductTotalPrice
    total_spent_per_client = orders.groupby('ClientUUId')['ProductTotalPrice'].sum().reset_index()
    
    # Переименование колонок для ясности
    total_spent_per_client.columns = ['ClientUUId', 'money_spent_total_sum']
    
    # Объединение признаков с `train` и `test`
    train = train.merge(total_spent_per_client, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(total_spent_per_client, on='ClientUUId', how='left').fillna(-1)
    

if UNIQUE_PRODUCTS_TOTAL_SUM:
    # Подсчет уникальных продуктов для каждого клиента
    unique_products = orders.groupby('ClientUUId')['ProductUUId'].nunique().reset_index()
    unique_products.columns = ['ClientUUId', 'unique_products_total_sum']
    
    # Объединение с train и test
    train = train.merge(unique_products, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_products, on='ClientUUId', how='left').fillna(-1)


if ORDER_TYPE_TOTAL_SUM:
    # Заказы любого типа
    order_type_sums = orders.pivot_table(index='ClientUUId', 
                                         columns='OrderType', 
                                         aggfunc='size', 
                                         fill_value=0).reset_index()
    # Переименование колонок для ясности
    order_type_sums.columns = ['ClientUUId'] + [f"order_type_{col}_total_sum" for col in order_type_sums.columns[1:]]
    
    # Объединение признаков с `train` и `test`
    train = train.merge(order_type_sums, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(order_type_sums, on='ClientUUId', how='left').fillna(-1)
    

if DAYS_SINCE_LAST_ORDER:
    promo_start_date = pd.Timestamp('2023-11-02')
        
    # Подсчет дней с момента последнего заказа до начала промо
    last_order_time = orders.groupby('ClientUUId')['Date'].max().reset_index()
    last_order_time['Date'] = pd.to_datetime(last_order_time['Date'])
    last_order_time['days_since_last_order'] = (promo_start_date - last_order_time['Date']).dt.days
    
    # Заполнение пропусков (например, значение `DaysSinceLastOrder` для клиентов, у которых не было заказов)
    last_order_time['days_since_last_order'].fillna(-1, inplace=True)
    
    # Объединение с train и test
    train = train.merge(last_order_time[['ClientUUId', 'days_since_last_order']], on='ClientUUId', how='left').fillna(-1)
    test = test.merge(last_order_time[['ClientUUId', 'days_since_last_order']], on='ClientUUId', how='left').fillna(-1)


if UNIQUE_CATEGORIES_TOTAL_SUM:
    # Подсчет количества уникальных категорий товаров для каждого клиента
    unique_categories_per_client = orders.groupby('ClientUUId')['CategoryId'].nunique().reset_index()
    unique_categories_per_client.columns = ['ClientUUId', 'unique_categories_total_sum']
    
    # Объединение с train и test
    train = train.merge(unique_categories_per_client, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_categories_per_client, on='ClientUUId', how='left').fillna(-1)


if PROMO_VALUE:
    # Вычисление "выгодности" промокода
    train['promo_value'] = train['Discount'] / train['OrderPrice']
    test['promo_value'] = test['Discount'] / test['OrderPrice']
    
    # Замена бесконечностей и NaN на 0 (если OrderPrice был 0 или отсутствовал)
    train['promo_value'].replace([float('inf'), -float('inf')], 0, inplace=True)
    test['promo_value'].replace([float('inf'), -float('inf')], 0, inplace=True)
    
    train['promo_value'].fillna(-1, inplace=True)
    test['promo_value'].fillna(-1, inplace=True)


if PROMO_USAGE_RATIO_TOTAL_SUM:
    # Группировка данных по ClientUUId и ClientOrderNumber
    grouped_orders = orders.groupby(['ClientUUId', 'ClientOrderNumber'])['apply_promo'].max().reset_index()
    
    # Подсчет общего количества заказов для каждого клиента
    total_orders = grouped_orders.groupby('ClientUUId').size().reset_index(name='TotalOrders')
    
    # Подсчет количества заказов с промокодом для каждого клиента
    promo_orders = grouped_orders[grouped_orders['apply_promo'] == 1].groupby('ClientUUId').size().reset_index(name='PromoOrders')
    
    # Объединение двух DataFrame для вычисления отношения
    promo_usage_ratio = pd.merge(total_orders, promo_orders, on='ClientUUId', how='left').fillna(0)
    promo_usage_ratio['promo_usage_ratio_total_sum'] = promo_usage_ratio['PromoOrders'] / promo_usage_ratio['TotalOrders']
    
    # Объединение с train и test
    train = train.merge(promo_usage_ratio[['ClientUUId', 'promo_usage_ratio_total_sum']], on='ClientUUId', how='left').fillna(-1)
    test = test.merge(promo_usage_ratio[['ClientUUId', 'promo_usage_ratio_total_sum']], on='ClientUUId', how='left').fillna(-1)

    

if EACH_ORDER_STATE_TOTAL_SUM:
    # Подсчет заказов с состоянием 11 и 4 для каждого клиента
    order_states_sum = orders.groupby(['ClientUUId', 'OrderState']).size().unstack(fill_value=0)
    order_states_sum.columns = [f"order_state_{int(col)}_total_sum" for col in order_states_sum.columns]
    
    # Объединение с train и test
    train = train.merge(order_states_sum, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(order_states_sum, on='ClientUUId', how='left').fillna(-1)
    

if DEVICE_DIVERSITY:
    # Подсчет уникальных типов устройств (Platform) для каждого клиента
    device_diversity = mobile_events.groupby('ClientUUId')['Platform'].nunique().reset_index()
    device_diversity.columns = ['ClientUUId', 'device_diversity']
    
    # Объединение с train и test
    train = train.merge(device_diversity, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(device_diversity, on='ClientUUId', how='left').fillna(-1)
    
    
if UNIQUE_SECTOR_TOTAL_SUM:
    # Подсчет уникальных секторов доставки (deliverySectorId) для каждого клиента
    unique_sector_count = orders.groupby('ClientUUId')['deliverySectorId'].nunique().reset_index()
    unique_sector_count.columns = ['ClientUUId', 'unique_sector_total_sum']
    
    # Объединение с train и test
    train = train.merge(unique_sector_count, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_sector_count, on='ClientUUId', how='left').fillna(-1)

if ORDER_PAYMENT_TYPE_SUM:
    # Подсчет использования каждого типа платежа
    payment_type_sum = orders.pivot_table(index='ClientUUId', 
                                          columns='OrderPaymentType', 
                                          aggfunc='size', 
                                          fill_value=0).reset_index()
    payment_type_sum.columns = ['ClientUUId', 'order_payment_type_0', 'order_payment_type_1', 'order_payment_type_2']
    
    # Объединение с train и test
    train = train.merge(payment_type_sum, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(payment_type_sum, on='ClientUUId', how='left').fillna(-1)
    
if AVG_TIME_BETWEEN_ORDERS:
    # Преобразование столбца Date в формат datetime
    orders['Date'] = pd.to_datetime(orders['Date'])
    
    # Группировка по ClientUUId и ClientOrderNumber и выбор минимальной даты для каждого заказа
    orders_min_date = orders.groupby(['ClientUUId', 'ClientOrderNumber'])['Date'].min().reset_index()
    
    # Сортировка по ClientUUId и Date для вычисления разницы во времени
    orders_sorted = orders_min_date.sort_values(['ClientUUId', 'Date'])
    orders_sorted['TimeDiff'] = orders_sorted.groupby('ClientUUId')['Date'].diff().dt.days
    
    # Вычисление среднего времени между заказами для каждого клиента
    avg_time_between_orders = orders_sorted.groupby('ClientUUId')['TimeDiff'].mean().reset_index()
    avg_time_between_orders.columns = ['ClientUUId', 'AvgTimeBetweenOrders']
    
    # Объединение с train и test
    train = train.merge(avg_time_between_orders, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(avg_time_between_orders, on='ClientUUId', how='left').fillna(-1)

    

if ORDERS_WEEKDAY_TOTAL_SUM:
    orders['Date'] = pd.to_datetime(orders['Date'], errors='coerce')
    orders['DayOfWeek'] = orders['Date'].dt.dayofweek
    orders_by_weekday = orders.groupby(['ClientUUId', 'DayOfWeek']).size().unstack(fill_value=0)
    orders_by_weekday.columns = [f'orders_weekday_{i}_total_sum' for i in range(7)]
    
    train = train.merge(orders_by_weekday, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(orders_by_weekday, on='ClientUUId', how='left').fillna(-1)
    

if MAX_ORDER_VALUE:
    max_order_value = orders.groupby('ClientUUId')['OrderTotalPrice'].max().reset_index()
    max_order_value.columns = ['ClientUUId', 'max_order_value']
    
    train = train.merge(max_order_value, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(max_order_value, on='ClientUUId', how='left').fillna(-1)
    
    
if AVG_DISCOUNT:
    # Группировка на уровне заказа для расчета общей цены по меню и общей цены с учетом скидок
    orders_grouped = orders.groupby(['ClientUUId', 'OrderUUId']).agg(
        MenuPriceSum=('MenuPrice', 'sum'),
        ProductTotalPriceSum=('ProductTotalPrice', 'sum')
    ).reset_index()
    
    # Расчет процентной скидки на уровне заказа
    orders_grouped['DiscountPercent'] = (
        (orders_grouped['MenuPriceSum'] - orders_grouped['ProductTotalPriceSum']) 
        / orders_grouped['MenuPriceSum'] * 100
    )
    
    # Учет случаев, когда MenuPrice может быть 0
    orders_grouped['DiscountPercent'].replace([np.inf, -np.inf], 0, inplace=True)
    
    # Вычисление среднего процента скидки для каждого клиента
    average_discount = orders_grouped.groupby('ClientUUId')['DiscountPercent'].mean().reset_index()
    average_discount.columns = ['ClientUUId', 'avg_discount']
    
    # Объединение с train и test
    train = train.merge(average_discount, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(average_discount, on='ClientUUId', how='left').fillna(-1)



if FIRST_LAST_ORDER_DIFF:
    orders_sorted = orders.sort_values(['ClientUUId', 'Date'])
    first_last_order_diff = orders_sorted.groupby('ClientUUId').agg({'Date': ['min', 'max']})
    first_last_order_diff.columns = ['FirstOrderDate', 'LastOrderDate']
    first_last_order_diff['DaysBetweenFirstAndLastOrder'] = (first_last_order_diff['LastOrderDate'] - first_last_order_diff['FirstOrderDate']).dt.days
    first_last_order_diff = first_last_order_diff.reset_index()
    train = train.merge(first_last_order_diff[['ClientUUId', 'DaysBetweenFirstAndLastOrder']], on='ClientUUId', how='left').fillna(-1)
    test = test.merge(first_last_order_diff[['ClientUUId', 'DaysBetweenFirstAndLastOrder']], on='ClientUUId', how='left').fillna(-1)
    

if PROMO_DISCOUNT_OCTOBER_SUM:
    promo_discount_total = cl_pr_oct.groupby('ClientUUId')['Discount'].sum().reset_index()
    promo_discount_total.columns = ['ClientUUId', 'promo_discount_total_sum']
    train = train.merge(promo_discount_total, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(promo_discount_total, on='ClientUUId', how='left').fillna(-1)
    
    
if PROMO_USAGE_RATIO_OCTOBER:
    orders['Date'] = pd.to_datetime(orders['Date'], errors='coerce')
    
    # Фильтрация данных за октябрь (с 1 по 31)
    start_date = pd.Timestamp('2023-10-01')
    end_date = pd.Timestamp('2023-10-31')
    october_orders = orders[(orders['Date'] >= start_date) & (orders['Date'] <= end_date)]
    
    # Группировка на уровне заказа для определения использования промокода
    orders_grouped = october_orders.groupby(['ClientUUId', 'ClientOrderNumber']).agg(
        ApplyPromo=('apply_promo', 'max'),
    ).reset_index()

    # Подсчет уникальных заказов с промокодом
    promo_orders_count = orders_grouped[orders_grouped['ApplyPromo'] == 1].groupby('ClientUUId').size().reset_index(name='PromoOrders')
    
    # Подсчет общего количества уникальных заказов
    total_orders_count = orders_grouped.groupby('ClientUUId').size().reset_index(name='TotalOrders')
    
    # Объединение данных и вычисление отношения
    promo_ratio = pd.merge(total_orders_count, promo_orders_count, on='ClientUUId', how='left').fillna(0)
    promo_ratio['promo_usage_ratio_october'] = promo_ratio['PromoOrders'] / promo_ratio['TotalOrders']
    
    # Объединение с train и test
    train = pd.merge(train, promo_ratio[['ClientUUId', 'promo_usage_ratio_october']], on='ClientUUId', how='left').fillna(-1)
    test = pd.merge(test, promo_ratio[['ClientUUId', 'promo_usage_ratio_october']], on='ClientUUId', how='left').fillna(-1)

    
if UNIQUE_PIZZERIA_TOTAL_SUM:
    # Подсчет уникальных id пиццерий (UnitUUId) для каждого клиента
    unique_sector_count = orders.groupby('ClientUUId')['UnitUUId'].nunique().reset_index()
    unique_sector_count.columns = ['ClientUUId', 'unique_pizzeria_total_sum']
    
    # Объединение с train и test
    train = train.merge(unique_sector_count, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_sector_count, on='ClientUUId', how='left').fillna(-1)
    
    
if CATEGORY_USAGE:
    # Выделяем заказы, в которых использовался промокод
    orders_with_promo = orders[orders['apply_promo'] == 1]
    
    # Подсчет количества заказов по категориям продуктов для каждого клиента
    category_usage = orders_with_promo.groupby(['ClientUUId', 'CategoryId']).size().unstack(fill_value=0)
    
    # Переименование колонок для ясности
    category_usage.columns = [f"category_{col}_promo_used" for col in category_usage.columns]
    
    # Объединение с train и test
    train = train.merge(category_usage, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(category_usage, on='ClientUUId', how='left').fillna(-1)
    
if HOURS_ORDERS:
    # Определяем функцию для классификации времени суток
    def classify_daytime(hour):
        if 6 <= hour < 12:
            return 'morning'
        elif 12 <= hour < 18:
            return 'day'
        elif 18 <= hour < 24:
            return 'evening'
        else:
            return 'night'
    
    # Применяем функцию к столбцу с датами заказов
    orders['OrderHour'] = pd.to_datetime(orders['Date']).dt.hour
    orders['Daytime'] = orders['OrderHour'].apply(classify_daytime)
    
    # Подсчет времени суток для каждого клиента
    daytime_counts = orders.groupby(['ClientUUId', 'Daytime']).size().unstack(fill_value=0)
    daytime_counts.columns = [f"{col}_orders" for col in daytime_counts.columns]
    
    # Объединение с train и test
    train = train.merge(daytime_counts, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(daytime_counts, on='ClientUUId', how='left').fillna(-1)


if TOTAL_DISCOUNTS:
    # Вычисляем сумму скидок для каждого заказа
    orders['DiscountAmount'] = orders['MenuPrice'] - orders['ProductTotalPrice']
    
    # Суммируем скидки для каждого клиента
    total_discounts = orders.groupby('ClientUUId')['DiscountAmount'].sum().reset_index()
    total_discounts.columns = ['ClientUUId', 'total_discount_amount']
    
    # Объединение с train и test
    train = train.merge(total_discounts, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(total_discounts, on='ClientUUId', how='left').fillna(-1)


if VISIT_TOKENS:
    # Подсчет уникальных VisitToken для каждого клиента
    unique_visit_tokens = mobile_events.groupby('ClientUUId')['VisitToken'].nunique().reset_index()
    unique_visit_tokens.columns = ['ClientUUId', 'unique_visit_tokens']
    
    # Объединение с train и test
    train = train.merge(unique_visit_tokens, on='ClientUUId', how='left').fillna(-1)
    test = test.merge(unique_visit_tokens, on='ClientUUId', how='left').fillna(-1)

In [ ]:
train.to_csv('my_train.csv', index=False)
test.to_csv('my_test.csv', index=False)